# 1-关系代数
    pd.merge() 实现的功能基于关系代数的一部分。
    关系代数是处理关系型数据的通用理论，绝大部分数据库的可用操作都以此为理论基础。
    关系代数方法论的强大之处在于，它提出的若干简单操作规则经过组合就可以为任意数据集构建十分复杂的操作。
    借助在数据库或程序里已经高效实现的基本操作规则，可以完成许多非常复杂的操作。
    
    Pandas 在 pd.merge() 函数与 Series 和 DataFrame 的 join() 方法里实现了这些基本操作规则。

In [1]:
import pandas as pd

# 2-数据连接的类型
    pd.merge() 函数实现了三种数据连接的类型：一对一、多对一和多对多。都是通过 pd.merge() 接口进行调用。

01--一对一连接

In [2]:
# 两个包含同一所公司员工不同信息的 DataFrame
# 用字典创建 DataFrame  
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'group':['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1);print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [3]:
# 将这两个 DataFrame 合并成一个 DataFrame，可以用 pd.merge() 函数实现
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


pd.merge() 方法会发现两个 DataFrame 都有“employee”列，并会自动以这列作为键进行连接。

两个输入的合并结果是一个新的 DataFrame。

pd.merge() 会默认丢弃原来的行索引，也可以自定义（下一节）。

02--多对一连接

    在需要连接的两个列中，有一列的值有重复。
    通过多对一连接获得的结果 DataFrame 将会保留重复值。

In [4]:
# 在结果 DataFrame 中多了一个“supervisor”列
# 里面有些值会因为输入数据的对应关系而有所重复。
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3);print(df4);
print(pd.merge(df3, df4))

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


03--多对多连接
    
    如果左右两个输入的共同列都包含重复值，那么合并的结果就是一种多对多连接。

In [5]:
# df5 显示不同岗位人员的一种或多种能力
# 可以得知每位员工所具备的能力
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 
                              'Engineering', 'Engineering', 'HR', 'HR'], 
                    'skills': ['math', 'spreadsheets', 'coding',
                               'linux', 'spreadsheets', 'organization']})
print(df1);print(df5);
print(pd.merge(df1, df5))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


# 3-设置数据合并的键
    pd.merge() 的默认行为：它会将两个输入的一个或多个共同列作为键进行合并。
    但由于两个输入要合并的列通常都不是同名的，因此 pd.merge() 提供了一些参数处理这个问题。

01--参数 on 的用法

    直接将参数 on 设置为一个列名字符串或者一个包含多列名称的列表.
    只能在两个 DataFrame 有共同列名的时候才可以使用

In [6]:
print(df1);print(df2);print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


02--left_on 与 right_on 参数

    有时也需要合并两个列名不同的数据集。
    例如前面的员工信息表中有一个字段不是“employee”而是“name”。
    可以用 left_on 和 right_on 参数来指定列名。
    
    会产生多余列，用 drop() 函数删除重复

In [7]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1);print(df3);
print(pd.merge(df1, df3, left_on='employee', right_on='name'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


获取的结果中会有一个多余的列，可以通过 DataFrame 的 drop() 方法将这列去掉

In [8]:
# 去除一列
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


03--left_index 与 right_index 参数
    
    除了合并列之外，还可合并索引

In [9]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a);print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


通过设置 pd.merge() 中的 left_index 或 right_index 参数将索引设置为键来实现合并。

In [10]:
# 合并索引
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


DataFrame 实现了 join() 方法，它可以按照索引进行数据合并

In [11]:
# join() 函数，按索引合并，同上
print(df1a.join(df2a))

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


将索引与列混合使用，可通过结合 left_index 与 right_on，或者结合 left_on 与 right_index 来实现。

In [12]:
print(df1a); print(df3);
print(pd.merge(df1a, df3, left_index=True, right_on='name'))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


# 4-设置数据连接的集合操作规则
    集合操作规则：
        当一个值出现在一列，却没有出现在另一列时，就需要考虑集合操作规则。

In [13]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
print(df6); print(df7); 
print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


默认情况下，结果中只会包含两个输入集合的交集，这种连接方式被称为内连接。

可以用 how 参数设置连接方式，默认值为 'inner'。

In [14]:
# 默认时，同上
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


how 参数支持的数据连接方式还有 'outer'、'left' 和 'right'

In [15]:
# 外连接（outer join）返回两个输入列的交集，所有缺失值都用 NaN 填充
print(pd.merge(df6, df7, how='outer'))

     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


In [16]:
# 左连接（left join）和右连接（right join）返回的结果分别只包含左列和右列
print(pd.merge(df6, df7, how='left'))
print(pd.merge(df6, df7, how='right'))

    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


# 5-重复列名： suffixes 参数
    两个输入 DataFrame 有重名列的情况

In [17]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
print(df8); print(df9);
print(pd.merge(df8, df9, on="name"))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


由于输出结果中有两个重复的列名，因此 pd.merge() 函数会自动为它们增加后缀 _x 或 _y

In [18]:
# 通过 suffixes 参数自定义后缀名
print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"]))

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


suffixes 参数同样适用于任何连接方式，即使有三个及三个以上的重复列名时也同样适用。

# 6-案例：美国各州的统计数据
    数据的合并与连接是组合来源不同的数据的最常用方法

In [19]:
# 用 Pandas 的 read_csv() 函数看看这三个数据集
pop = pd.read_csv('../Data/state-population.csv')
areas = pd.read_csv('../Data/state-areas.csv')
abbrevs = pd.read_csv('../Data/state-abbrevs.csv')

# .head() 输出前五条输出，对应 .tail() 输出最后五条数据
print(pop.head());
print(areas.head());
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


看过这些数据之后，我们想要计算一个比较简单的指标：美国各州的人口密度排名。

虽然可以直接通过计算每张表获取结果，但这次试着用数据集连接来解决这个问题。

In [20]:
# 用一个多对一合并获取人口（pop）DataFrame 中各州名称缩写对应的全称.
# 将 pop 的 state/region 列与 abbrevs 的abbreviation 列进行合并，
# 需要通过 how='outer' 确保数据没有丢失。
merged = pd.merge(pop, abbrevs , how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1)  # 丢弃重复信息

merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [21]:
# 检查一下数据是否有缺失，可以对每个字段逐行检查是否有缺失值
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [22]:
# 部分 population 是缺失值，检查是哪些数据
# 花哨的索引
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [23]:
# 还发现一些新的州的数据也有缺失，可能是由于名称缩写没有匹配上全程！
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [24]:
# 可以快速解决这个问题：人口数据中包含波多黎各（PR）和全国总数（USA），
# 但这两项没有出现在州名称缩写表中，快速填充对应的全称。
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico' 
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States' 
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [25]:
# 用类似的规则将面积数据也合并进来。用两个数据集共同的 state 列来合并
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [26]:
# 再检查一下数据，看看哪些列还有缺失值，没有匹配上
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [27]:
# 面积 area 列里面还有缺失值。来看看究竟是哪些地区面积缺失
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [28]:
# 发现面积（areas）DataFrame 里面不包含全美国的面积数据。
# 可以插入全国总面积数据（对各州面积求和即可），
# 但是针对本案例，我们要去掉这个缺失值，因为全国的人口密度在此无关紧要
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [29]:
# 再次检查，发现无缺失
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [30]:
# 先选择 2010 年的各州人口以及总人口数据
# 用 query() 函数进行快速计算(见3.13)
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [31]:
# 计算人口密度并按序排列
# 对索引进行重置，然后再计算结果
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [32]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [33]:
# 人口密度最低的几个州的数据
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64